In [1]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from transformer.date_transformer import DateTransformer
from transformer.column_transformer import ColumnTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math
from IPython.core.debugger import set_trace

In [2]:
## Loading Constants
start = "2008-01-01"
end = datetime(2021,1,1).strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("unity")
market = Market()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
quarterly_range = range(1,5)
yearly_range = range(2014,2021)
model_range = range(len(tickers["Symbol"]))
datasets = ["pdr"
#             ,"tiingo"
#             ,"finnhub"
           ]

In [3]:
sims = []
gap = 365
year_gap = int(gap/365)
quarter_gap = int(gap/90)
training_years = 4
training_days = 100
sec.connect()
strat_db.connect()
market.connect()
for dataset in datasets:
#     strat_db.drop_table("{}_fundamental_quarterly_sim".format(dataset))
    strat_db.drop_table("{}_fundamental_quarterly_sim_errors")
    quarterly_classification_data = market.retrieve_data("dataset_{}_quarter_classification".format(dataset))
    quarterly_regression_data = market.retrieve_data("dataset_{}_quarter_regression".format(dataset))
#     strat_db.drop_table("{}_price_quarterly_sim".format(dataset))
    strat_db.drop_table("{}_price_quarterly_sim_errors".format(dataset))
    for year in tqdm(yearly_range):
        for quarter in tqdm(quarterly_range):
            try:
                ## Setting Up
                print(year,quarter)
                date_ranges = DateUtils.create_quarterly_training_range_rec(year,quarter,training_years,gap)
                training_start,training_end,prediction_start,prediction_end = date_ranges
                dates = pd.to_datetime(date_ranges)
                quarters = [x.quarter for x in dates]
                years = [x.year for x in dates]
                print(date_ranges)
                ### switch to all tickers
                for i in model_range:
                    try:
                        ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                        if ticker in quarterly_regression_data.columns:
                            cik = int(tickers.iloc[i]["CIK"].item())
                            price = market.retrieve_price_data("{}_prices".format(dataset),ticker)
                            if dataset == "pdr":
                                price = ColumnTransformer.rename_columns(price," ")
                            else:
                                price = ColumnTransformer.rename_columns(price,"")
                            price = DateTransformer.convert_to_date(dataset,price,"date")
                            filing = sec.retrieve_filing_data(cik)
                            mt = ModelTransformer(ticker,training_start,training_end,gap)
                            pt = ModelTransformer(ticker,prediction_start,prediction_end,gap)
                            mr = ModelPreprocessor(ticker)
                            prot = ProductTransformer(ticker,prediction_start,prediction_end)
                            ## regression_model
                            rfd = mt.fundamental_merge(price.copy(),filing.copy(),True,classify=False)
                            factors = rfd.columns 
                            refined = mr.fundamental_preprocess(rfd.copy())
                            rfr = sp.regression(refined,ranked=False,tf=False)
                            cfd = mt.fundamental_merge(price.copy(),filing.copy(),True,classify=True)
                            refined = mr.fundamental_preprocess(cfd.copy())
                            cfr = sp.classification(refined,tf=False)
                            fundamental_results = pd.DataFrame([cfr
                                                                ,rfr])
                            product_refineds = []
                            product_qpds = []
                            for i in range(len(fundamental_results)):
                                fundamental_result = fundamental_results.iloc[i]
                                if fundamental_result["model_type"] == "regression":
                                    fundamental_data = pt.fundamental_merge_rec(filing.copy(),False,factors)
                                else:
                                    fundamental_data = pt.fundamental_merge_rec(filing.copy(),True,factors)
                                product_refined = mr.fundamental_preprocess_rec(fundamental_data.copy())
                                product_refineds.append(product_refined)
                            sim = prot.merge_quarterlies_v2(product_refineds,fundamental_results,year,quarter,"fundamental",gap)
                            strat_db.store_data("{}_fundamental_quarterly_sim".format(dataset),pd.DataFrame([sim]))
                    except Exception as e:
                        message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                        print(message)
                        strat_db.store_data("fundamental_quarterly_sim_errors",pd.DataFrame([message]))
            except Exception as e:
                print(year,quarter,str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                                                | 0/4 [00:00<?, ?it/s]

2014 1
['2010-01-01', '2013-12-31', '2014-01-01', '2014-03-31']
[13:01:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
{'status': 'quarterly modeling', 'ticker': 'ALB', 'year': '2014', 'quarter': '1', 'message': 'at least one array or dtype is required'}
{'status': 'quarterly modeling', 'ticker': 'ALLE', 'year': '2014', 'quarter': '1', 'message': 'With n_samples=1, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'quarterly modeling', 'ticker': 'AMC', 'year': '2014', 'quarter': '1', 'message': 'With n_samples=1, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.'}
{'status': 'quarterly modeling', 'ticker': 'AMCR', 'year': '2014', 'quarter': '1', 'message': 'single positional indexer is out-of-bounds'}
[13:02:04] WARNING: C:/Users/Administrator/workspace/xgboost-w

{'status': 'quarterly modeling', 'ticker': 'KHC', 'year': '2014', 'quarter': '1', 'message': 'at least one array or dtype is required'}
[13:21:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
[13:21:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
[13:22:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
[13:22:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
{'status': 'quarterly modeling', 'ticker': 'LW', 'year': '2014', 'quarter': '1', 'message': 'Found array with 0 sample(s) (shape=(0, 83)) while a minimum of 1 is required.'}
[13:23:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
[13:23:13] WARNING: C:/Users/Administrator/wo


 25%|█████████████████████████████████                                                                                                   | 1/4 [41:14<2:03:42, 2474.21s/it]

{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2014', 'quarter': '1', 'message': 'at least one array or dtype is required'}
2014 2
['2010-04-01', '2014-03-31', '2014-04-01', '2014-06-30']
{'status': 'quarterly modeling', 'ticker': 'ALB', 'year': '2014', 'quarter': '2', 'message': 'at least one array or dtype is required'}
{'status': 'quarterly modeling', 'ticker': 'ALLE', 'year': '2014', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}
{'status': 'quarterly modeling', 'ticker': 'AMC', 'year': '2014', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}
[13:43:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
{'status': 'quarterly modeling', 'ticker': 'ANET', 'year': '2014', 'quarter': '2', 'message': 'Found array with 0 sample(s) (shape=(0, 85)) while a minimum of 1 is required.'}
{'status': 'quarterly modeling', 'ticker': 'APA', 'year': '2014', 'quarter': '2'

...

{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2020', 'quarter': '2', 'message': 'at least one array or dtype is required'}
2020 3
['2016-07-01', '2020-06-30', '2020-07-01', '2020-09-30']
[04:52:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
{'status': 'quarterly modeling', 'ticker': 'ALB', 'year': '2020', 'quarter': '3', 'message': 'at least one array or dtype is required'}
{'status': 'quarterly modeling', 'ticker': 'APA', 'year': '2020', 'quarter': '3', 'message': 'at least one array or dtype is required'}
[04:55:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
{'status': 'quarterly modeling', 'ticker': 'BRK-B', 'year': '2020', 'quarter': '3', 'message': 'at least one array or dtype is required'}
{'status': 'quarterly modeling', 'ticker': 'CARR', 'year': '2020', 'quarter': '3', 'message': 'single positional indexer is out-of-b


 75%|███████████████████████████████████████████████████████████████████████████████████████████████████                                 | 3/4 [1:28:05<29:21, 1761.23s/it]

{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2020', 'quarter': '3', 'message': 'at least one array or dtype is required'}
2020 4
['2016-10-03', '2020-09-30', '2020-10-01', '2020-12-31']
[05:22:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
{'status': 'quarterly modeling', 'ticker': 'ALB', 'year': '2020', 'quarter': '4', 'message': 'at least one array or dtype is required'}
{'status': 'quarterly modeling', 'ticker': 'APA', 'year': '2020', 'quarter': '4', 'message': 'at least one array or dtype is required'}
[05:25:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1190: Empty dataset at worker: 0
{'status': 'quarterly modeling', 'ticker': 'BRK-B', 'year': '2020', 'quarter': '4', 'message': 'at least one array or dtype is required'}
{'status': 'quarterly modeling', 'ticker': 'CCL', 'year': '2020', 'quarter': '4', 'message': 'at least one array or dtype is require


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [16:51:43<00:00, 8671.88s/it]

{'status': 'quarterly modeling', 'ticker': 'ZTS', 'year': '2020', 'quarter': '4', 'message': 'at least one array or dtype is required'}
